In [1]:
# 1. Post 중 자신에게 포함된 Comment들 중에
# content 속성(column)에 '좋아요' 이라는 문자열이 들어가는 경우인 Post 목록
#     - 일부 Post가 '좋아요' 이라는 내용을 포함하는 댓글을 가져야 함
# 1-1. Post 중 태그가 없는 경우인 Post 목록

In [2]:
Post.objects.filter(
    postcomment__content__contains='좋아요'
).values('pk')

<QuerySet [{'pk': 6}, {'pk': 6}, {'pk': 6}]>

In [3]:
Post.objects.filter(
    tags__isnull=True
).values_list('pk', flat=True).order_by('-pk')

<QuerySet [14, 8, 6, 4, 3, 2, 1]>

In [4]:
# 2. Post 중 자신에게 포함된 Comment들 중에
# Post의 author와 Comment의 author가 다른 경우인 Comment가 존재하는 Post 목록
#     > 댓글은 존재하나, 작성자가 작성한 댓글은 없는 경우인 Post 목록
#     - Post 작성자 외 다른 사용자가 댓들을 단 경우가 존재해야 함
#     - Django의 F Expression 사용

In [5]:
Post.objects.exclude(
    postcomment__isnull=True
).exclude(
    postcomment__author=F('author')
).values_list('pk', flat=True).order_by('-pk')

<QuerySet []>

In [6]:
# 3. HashTag 중 자신에게 연결된 Comment의 Post의 author의 pk가 1인 경우인 HashTag 목록
#     - 없다면 author의 pk를 바꿔봄

In [8]:
# tag = Tag.objects.first()
Tag.objects.filter(
    posts__author__pk=1
).values_list('posts', flat=True).distinct()

<QuerySet [5, 7, 9, 10, 11, 12, 13]>

In [ ]:
# 4. Post 중 자신의 좋아요 개수가 1개 이상인(좋아요가 존재하는) Post 목록
#     - Field lookup의 'isnull' 항목 참조

In [ ]:
Post.objects.filter(
    postlike__isnull=False
)

In [ ]:
Post.objects.exclude(
    postlike__isnull=True
).values_list('pk', flat=True)

In [ ]:
# 5. Comment 중 자신의 Post의 좋아요 개수가 1개 이상인 Comment 목록
#     - Field lookup의 'isnull' 항목 참조

In [ ]:
PostComment.objects.filter(
    post__postlike__isnull=False
).values_list('pk', flat=True).distinct()

In [ ]:
PostComment.objects.filter(
    post__postlike__isnull=False
).values_list('pk', 'post', 'post__postlike', 'post__postlike__user')

In [ ]:
# 6. Comment 중 자신의 Post에 속하는 가장 최근 PostLike가 1주일 이내인 Comment 목록
#     - from django.utils import timezone
#     - 현재 시간은 timezone.now()로 가져올 수 있음

In [ ]:
from datetime import timedelta

now = timezone.now()
a_week_ago = now - timedelta(days=7)

Post.objects.filter(
    postlike__created__gte=a_week_ago
).values('pk')

In [ ]:
# Comment 중 자신의 Post에 속하는 가장 최근 PostLike가 3일 이후인 Comment 목록
Post.objects.filter(
    postlike__created__lt=a_week_ago
).values('pk', 'postlike__user')

In [ ]:
PostComment.objects.filter(
    post__postlike__created__lt=a_week_ago
).values('pk', 'post')

In [ ]:
a_week_ago = now - timedelta(hours=2)

Post.objects.filter(
    postlike__created__lt=a_week_ago
).values('pk', 'postlike__user')

for item in PostComment.objects.filter(
    post__postlike__created__lt=a_week_ago
).values('pk', 'post', 'post__postlike__created').order_by('-post__pk').filter(
    post__pk=21
):
    print(item)

In [ ]:
# 7. 각각의 Post에 댓글이 몇 개 달렸는지 QuerySet.values()로 꺼내보기
#     Django annotate / aggregate
#     QueyrSetAPI의 Aggregation functions의 Count()를 사용해야 함
#         -> QueyrSet.count()와 다름
#     Post.objects.annotate(<무언가>).values(<왼쪽에서한걸활용>)

In [ ]:
post = Post.objects.last()
post.postcomment_set.count()

In [ ]:
Post.objects.annotate(
    comment_count=Count('postcomment')
).values_list('pk', 'comment_count')

In [ ]:
# 8. 댓글이 3개 이상인 Post 목록만 가져오기
#     Post.objects.annotate(<무언가>).filter(<왼쪽에서한걸활용>)

In [ ]:
Post.objects.annotate(
    comment_count=Count('postcomment')
).filter(
    comment_count__gte=3
).values_list('pk', flat=True)

In [ ]:
# 9. 댓글이 3개 이상이며, 댓글 개수가 좋아요 개수보다 많은 Post 목록만 가져오기
#     Post.objects.annotate(<무언가>).filter(<왼쪽에서한걸활용>)

In [ ]:
Post.objects.annotate(
    comment_count=Count('postcomment'),
    like_count=Count('postlike')
).filter(
    comment_count__gte=3
#     comment_count__gt=F('like_count')
).values_list('pk', flat=True)